## Contexte & Chargement initial

Ce notebook vise à explorer et structurer les données d’accidents/incidents ferroviaires aux USA pour isoler un module "risque ferroviaire" exploitable dans une analyse globale de résilience logistique.

| Variable                       | Description courte                                                          | Type           | Exemples                                         |
|--------------------------------|----------------------------------------------------------------------------|----------------|--------------------------------------------------|
| Accident_Number                | Identifiant unique de l’accident                                           | object         | 'N1700013', '201707188', '0420001'               |
| Reporting_Railroad_Code        | Code société ferroviaire déclarante                                        | object         | 'NICD', 'NIRC', 'CR', 'IC', 'NS', 'UP'           |
| Reporting_Railroad_Name        | Nom société ferroviaire déclarante                                         | object         | 'Northern Indiana Commuter Transportation District' |
| Accident_Year                  | Année de l’accident                                                        | int64          | 2017, 1981                                       |
| Accident_Month                 | Mois de l’accident                                                         | int64          | 6, 4, 10, 12                                     |
| Day                            | Jour du mois                                                               | int64          | 6, 11, 7, 15                                     |
| Date                           | Date complète (mm/dd/yyyy)                                                 | object (date)  | '06/06/2017', '01/11/2007'                       |
| Time                           | Heure de l’accident                                                        | object         | '2:14 PM', '7:20 AM', '3:55 AM'                  |
| Accident_Type                  | Type d’accident                                                            | object         | 'Derailment', 'Side collision', 'Obstruction'    |
| State_Abbreviation             | Code état USA                                                              | object         | 'IL', 'LA', 'NY'                                 |
| State_Name                     | Nom de l’état                                                              | object         | 'ILLINOIS', 'LOUISIANA', 'NEW YORK'              |
| County_Name                    | Nom du comté                                                               | object         | 'COOK', 'KANKAKEE', 'ST JOHN THE BAPTIST'        |
| Latitude                       | Latitude de l’événement                                                    | float64        | 41.884034, 41.054254, 41.267569                  |
| Longitude                      | Longitude de l’événement                                                   | float64        | -87.623002, -87.906205, -88.207738               |
| Weather_Condition              | Conditions météo au moment de l’accident                                   | object         | 'Clear', 'Snow', 'Cloudy', 'Rain'                |
| Temperature                    | Température au moment de l’accident (°F)                                   | float64        | 65, 28, 56, 60                                   |
| Visibility                     | Conditions de visibilité                                                   | object         | 'Day', 'Dawn', 'Dark'                            |
| Total_Damage_Cost              | Coût total des dégâts (USD)                                                | float64        | 132013, 9986, 148289                             |
| Equipment_Damage_Cost          | Dégâts matériels sur le matériel roulant                                   | float64        | 98627, 5220, 7418, 15061                         |
| Track_Damage_Cost              | Dégâts sur l’infrastructure (rails)                                        | float64        | 0, 4766, 140871, 0                               |
| Primary_Accident_Cause         | Cause principale (libellé)                                                 | object         | 'Switch point worn or broken', 'Highway user deliberately disregarded crossing warning devices' |
| Primary_Accident_Cause_Code    | Code de la cause principale                                                | object         | 'T314', 'M308', 'H306', 'M404'                   |
| Accident_Cause                 | Cause d’accident (secondaire ou globale, libellé)                          | object         | 'Switch point worn or broken', 'Highway user deliberately disregarded crossing warning devices' |
| Accident_Cause_Code            | Code cause d’accident                                                      | object         | 'T314', 'M308', 'H306', 'M404'                   |
| Railroad_Employees_Killed      | Nombre d’employés tués                                                     | int64          | 0, 1                                             |
| Railroad_Employees_Injured     | Nombre d’employés blessés                                                  | int64          | 0, 5, 2                                          |
| Passengers_Killed              | Nombre de passagers tués                                                   | int64          | 0                                                |
| Passengers_Injured             | Nombre de passagers blessés                                                | int64          | 0, 14, 34                                        |
| Others_Killed                  | Nombre de tiers tués                                                       | int64          | 0, 3                                             |
| Others_Injured                 | Nombre de tiers blessés                                                    | int64          | 0, 1, 2                                          |
| Total_Persons_Killed           | Total tués (toutes catégories)                                             | int64          | 0, 3, 1                                          |
| Total_Persons_Injured          | Total blessés (toutes catégories)                                          | int64          | 0, 5, 34                                         |
| Hazmat_Cars                    | Nombre de wagons matières dangereuses présents                             | int64          | 0, 7                                             |
| Hazmat_Cars_Damaged            | Nombre de wagons matières dangereuses endommagés                           | int64          | 0, 1                                             |
| Hazmat_Released_Cars           | Nombre de wagons matières dangereuses avec fuite                           | int64          | 0                                                |
| Persons_Evacuated              | Nombre de personnes évacuées                                               | int64          | 0, 149                                           |
| Derailed_Loaded_Freight_Cars   | Nombre de wagons marchandises chargés déraillés                            | int64          | 0, 5                                             |
| Derailed_Empty_Freight_Cars    | Nombre de wagons marchandises vides déraillés                              | int64          | 0, 3                                             |
| Derailed_Loaded_Passenger_Cars | Nombre de voitures voyageurs chargées déraillées                           | int64          | 0                                                |
| Derailed_Empty_Passenger_Cars  | Nombre de voitures voyageurs vides déraillées                              | int64          | 0                                                |

In [ ]:
# Imports principaux 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
from sklearn.preprocessing import MinMaxScaler

# Affichage lisible
pd.set_option('display.max_columns', None)

# Chemin du fichier
DATA_PATH = '../data/extracted/Railroad_Accident_Incident_Data/Rail_Equipment_Accident_Incident_Data.csv'

In [ ]:
# Chargement du dataset
df = pd.read_csv(DATA_PATH)

# Shape du dataset
print(f"Shape du dataset : {df.shape}")

# Liste des colonnes
print("\nColonnes du dataset :")
print(df.columns.tolist())

# Aperçu des 3 premières lignes
display(df.head(3))

## Types, valeurs manquantes et sélection des variables clés

Objectif :
- Cartographier les types et les taux de valeurs manquantes pour toutes les colonnes.
- Préparer une shortlist robuste des variables à extraire pour le module “risque ferroviaire”.
- Repérer d’éventuelles colonnes à écarter (trop vides/inutiles), mais sans perdre les infos critiques pour la future analyse dashboard.

In [ ]:
# 1. Affichage synthétique des types et valeurs manquantes (trié décroissant)
types_na = pd.DataFrame({
    "Type": df.dtypes,
    "Nb_NA": df.isna().sum(),
    "Pourc_NA": (df.isna().mean() * 100).round(2)
}).sort_values("Pourc_NA", ascending=False)

display(types_na.head(25))  # Top 25 colonnes les plus vides
display(types_na.tail(15))  # 15 colonnes les plus remplies

# 2. Affichage des colonnes qui dépassent un seuil critique de valeurs manquantes (exemple 95%)
seuil_na = 95
cols_drop = types_na[types_na["Pourc_NA"] > seuil_na].index.tolist()
print(f"\nColonnes avec >{seuil_na}% de NaN ({len(cols_drop)}):\n{cols_drop}")

# 3. Shortlist des variables prioritaires à garder (adaptée pour dashboard risque)
variables_prio = [
    # Structure/identité
    "Accident Number", "Reporting Railroad Code", "Reporting Railroad Name",
    "Accident Year", "Accident Month", "Day", "Date", "Time", "Report Year",
    # Nature et contexte
    "Accident Type", "State Abbreviation", "State Name", "County Name", "Latitude", "Longitude",
    "Weather Condition", "Temperature", "Visibility", "Train Speed",
    # Impact/coût
    "Total Damage Cost", "Equipment Damage Cost", "Track Damage Cost",
    # Causes
    "Primary Accident Cause", "Primary Accident Cause Code",
    "Accident Cause", "Accident Cause Code",
    # Humain
    "Railroad Employees Killed", "Railroad Employees Injured",
    "Passengers Killed", "Passengers Injured",
    "Others Killed", "Others Injured", "Total Persons Killed", "Total Persons Injured",
    # Risque/impact logistique
    "Hazmat Cars", "Hazmat Cars Damaged", "Hazmat Released Cars", "Persons Evacuated",
    "Derailed Loaded Freight Cars", "Derailed Empty Freight Cars",
    "Derailed Loaded Passenger Cars", "Derailed Empty Passenger Cars"
]

# Si certaines colonnes shortlistées sont absentes (orthographe, warning typique), on les liste :
missing = [v for v in variables_prio if v not in df.columns]
if missing:
    print("\nVariables shortlistées absentes dans le DataFrame :", missing)

print("\nVariables shortlistées pour la suite (pour dashboard) :")
print([v for v in variables_prio if v in df.columns])

## Nettoyage, typage et structuration des variables-clés

Objectif :
- Nettoyer et convertir chaque variable shortlistée au bon format pour l’analyse de risque.
- Gérer les valeurs manquantes et les incohérences (coûts, dates, chiffres…).
- Créer des variables supplémentaires utiles à l’analyse, comme la tranche horaire d’accident ("TimeOfDay").
- Préparer un DataFrame propre prêt pour l’agrégation, la data viz et le scoring.

In [ ]:
# Suppression des colonnes trop vides (aucun impact sur ta shortlist)
cols_drop = ['Adjunct Code Name 3', 'Adjunct Code 3', 'Adjunct Code Name 2', 'Adjunct Code 2', 'Other Railroad Company Grouping']
df_clean = df.drop(columns=cols_drop)

# Extraction des variables prioritaires (liste vérifiée précédemment)
variables_finales = [v for v in variables_prio if v in df_clean.columns]
df_risk = df_clean[variables_finales].copy()

# Nettoyage des colonnes "cost" (retrait des virgules, conversion en float, NA->0)
for col in ["Total Damage Cost", "Equipment Damage Cost", "Track Damage Cost"]:
    df_risk[col] = (
        df_risk[col]
        .astype(str)
        .str.replace(",", "", regex=False)
        .replace("nan", "0")
        .replace("", "0")
        .astype(float)
        .fillna(0)
    )

# Conversion des années/mois/jour en int, gestion des floats/residus NaN
for col in ["Accident Year", "Accident Month", "Day", "Report Year"]:
    if col in df_risk.columns:
        df_risk[col] = pd.to_numeric(df_risk[col], errors="coerce").fillna(0).astype(int)

# Conversion latitude/longitude
for col in ["Latitude", "Longitude"]:
    if col in df_risk.columns:
        df_risk[col] = pd.to_numeric(df_risk[col], errors="coerce")

# Conversion "Train Speed" en float
if "Train Speed" in df_risk.columns:
    df_risk["Train Speed"] = pd.to_numeric(df_risk["Train Speed"], errors="coerce")

# Nettoyage des variables "impact" (tués/blessés, wagons déraillés, etc.) en int (NA→0)
impact_cols = [
    "Railroad Employees Killed", "Railroad Employees Injured",
    "Passengers Killed", "Passengers Injured", "Others Killed", "Others Injured",
    "Total Persons Killed", "Total Persons Injured",
    "Hazmat Cars", "Hazmat Cars Damaged", "Hazmat Released Cars", "Persons Evacuated",
    "Derailed Loaded Freight Cars", "Derailed Empty Freight Cars",
    "Derailed Loaded Passenger Cars", "Derailed Empty Passenger Cars"
]
for col in impact_cols:
    if col in df_risk.columns:
        df_risk[col] = pd.to_numeric(df_risk[col], errors="coerce").fillna(0).astype(int)

# Création de la variable TimeOfDay à partir de 'Time'
def set_time_class(val):
    try:
        time, abbr = str(val).strip().split()
        hr = int(time.split(':')[0])
        if abbr.upper() == "AM":
            if hr < 6:
                return "EARLY MORNING"
            else:
                return "LATE MORNING"
        if abbr.upper() == "PM":
            if hr < 4:
                return "AFTERNOON"
            else:
                return "EVENING"
    except:
        return np.nan
df_risk["TimeOfDay"] = df_risk["Time"].apply(set_time_class)

# Nettoyage des chaînes : trim espaces/NA pour certains champs texte
for col in ["Accident Type", "State Abbreviation", "State Name", "County Name", "Weather Condition", "Visibility"]:
    if col in df_risk.columns:
        df_risk[col] = df_risk[col].astype(str).str.strip().replace("nan", "")

# Vérification finale : shape, types, et aperçu
print("Shape DataFrame final :", df_risk.shape)
display(df_risk.info())
display(df_risk.head(3))

## Analyse statistique et visualisation des risques

Objectif :
- Fournir une vision synthétique de la répartition, de la gravité et des caractéristiques des accidents.
- Générer des graphiques exploitables pour un dashboard d’analyse des risques logistiques.
- Mettre en avant les variables les plus contributives au “risque” (type, lieu, coût, météo, impact humain…).

In [ ]:
# On garde uniquement les accidents à partir de 2002 (pour avoir les 20 dernières années)
df_risk = df_risk[df_risk["Report Year"] >= 2002].copy()
print(f"Nombre d'accidents à partir de 2002 : {df_risk.shape[0]}")

# Nettoyage
df_risk["Accident Type"] = df_risk["Accident Type"].astype(str).str.strip()
df_risk = df_risk[
    df_risk["Accident Type"].notna() &
    (df_risk["Accident Type"] != "") &
    (df_risk["Accident Type"].str.lower() != "nan")
]

In [ ]:
# 1. Distribution des accidents par année
plt.figure(figsize=(12,4))
sns.countplot(x="Report Year", data=df_risk, order=sorted(df_risk["Report Year"].unique()))
plt.title("Nombre d'accidents par an")
plt.xticks(rotation=90)
plt.show()

# 2. Top 10 types d'accident
plt.figure(figsize=(12,4))
order = df_risk["Accident Type"].value_counts().index[:10]
sns.countplot(y="Accident Type", data=df_risk, order=order)
plt.title("Top 10 des types d'accident ferroviaire")
plt.show()

# 3. Distribution des coûts (log-scale)
plt.figure(figsize=(8,4))
sns.histplot(df_risk["Total Damage Cost"], bins=100, log_scale=True)
plt.title("Distribution des coûts totaux des accidents (échelle log)")
plt.xlabel("Coût total (USD)")
plt.show()


# 4. Accidents par conditions météo
plt.figure(figsize=(12,4))
order = df_risk["Weather Condition"].value_counts().index[:8]
sns.countplot(y="Weather Condition", data=df_risk, order=order)
plt.title("Accidents par conditions météo")
plt.show()

# 5. Accidents par tranche horaire (TimeOfDay)
plt.figure(figsize=(8,4))
sns.countplot(x="TimeOfDay", data=df_risk, order=["EARLY MORNING","LATE MORNING","AFTERNOON","EVENING"])
plt.title("Accidents par tranche horaire")
plt.show()


In [ ]:
# Evolution annuelle du nombre de tués et blessés 
df_risk_valid = df_risk[df_risk["Report Year"] > 1900] 
df_risk_yearly = df_risk_valid.groupby("Report Year")[["Total Persons Killed", "Total Persons Injured"]].sum()
display(df_risk_yearly)

ax = df_risk_yearly.plot(
    title="Evolution annuelle des tués/blessés",
    figsize=(8,5)
)
ax.set_xlabel("Report Year")
ax.set_ylabel("Total")
ax.xaxis.set_major_locator(ticker.MaxNLocator(integer=True))

years = df_risk_yearly.index
ax.set_xlim(years.min() - 1, years.max() + 1)

plt.show()

In [ ]:
# Croisement entre type d’accident et conditions météo sur la gravité humaine
cross = df_risk.groupby(['Accident Type', 'Weather Condition'])[
    ['Total Persons Killed', 'Total Persons Injured', 'Total Damage Cost']
].mean().unstack().fillna(0)

display(cross)

# Visualisation boxplot des coûts par type d’accident

# Vérifier et filtrer les types d'accident avec assez de data
type_counts = df_risk["Accident Type"].value_counts()
types_valides = type_counts[type_counts > 10].index  # ou adapte le seuil si besoin

df_plot = df_risk[
    (df_risk["Accident Type"].isin(types_valides)) &
    (df_risk["Total Damage Cost"] > 0) &
    (df_risk["Total Damage Cost"] < 5e5)  # Filtre pour la lisibilité
].copy()

plt.figure(figsize=(12,8))
sns.boxplot(
    x="Accident Type",
    y="Total Damage Cost",
    data=df_plot,
    showfliers=False,
    order=types_valides  # Assure le bon ordre
)
plt.title("Distribution du coût par type d'accident")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
accident_counts = df_risk["Accident Type"].value_counts(normalize=True) * 100
print(accident_counts)

## Hypothèse sur le volume annuel de trajets de trains aux États-Unis

L’estimation du risque relatif de chaque type d’incident ferroviaire repose sur une fourchette réaliste du **nombre total de trajets de trains par an** aux États-Unis.

Selon l’**Association of American Railroads** ([AAR Data Center](https://www.aar.org/data-center/)),  
le volume annuel de circulations de trains (fret + passagers) n’est pas publié précisément, mais peut être **estimé** entre **10 millions et 20 millions de trajets de trains par an** pour la période récente (2002-2022).  
Cela couvre l’ensemble des compagnies de fret, de passagers interurbains (Amtrak), et de banlieue.

Pour chaque type d’accident, nous calculons donc les probabilités et taux de survenue :
- **Sur la base d’un total de 210 millions de trajets (10M/an x 21 ans)**
- **Et d’un total de 420 millions de trajets (20M/an x 21 ans)**

Cette approche permet d’encadrer l’incertitude liée à l’absence de donnée consolidée, tout en fournissant une analyse robuste et réaliste du risque ferroviaire à l’échelle nationale.

> _Les résultats (taux/probabilités) sont donc exprimés sous forme de fourchette, reflétant cette incertitude._

In [ ]:
# Paramètres réalistes
min_trains_per_year = 10_000_000
max_trains_per_year = 20_000_000
n_years = 2022 - 2002 + 1

total_trains_min = min_trains_per_year * n_years
total_trains_max = max_trains_per_year * n_years

# Préparation summary complet par type d'accident
summary = df_risk_valid.groupby("Accident Type").agg(
    Incidents_recensés=("Accident Number", "count"),
    Coût_moyen_USD=("Total Damage Cost", "mean"),
    Total_tués=("Total Persons Killed", "sum"),
    Total_blessés=("Total Persons Injured", "sum")
)

summary["Proba_brute_basse_%"] = (summary["Incidents_recensés"] / total_trains_min * 100).round(8)
summary["Proba_brute_haute_%"] = (summary["Incidents_recensés"] / total_trains_max * 100).round(8)
summary["Tués_par_incident"] = (summary["Total_tués"] / summary["Incidents_recensés"]).round(3)
summary["Blessés_par_incident"] = (summary["Total_blessés"] / summary["Incidents_recensés"]).round(3)

# Incidents majeurs
major = df_risk_valid[df_risk_valid["Total Damage Cost"] > 1_000_000]["Accident Type"].value_counts()
summary["% incidents >1M$"] = ((major / summary["Incidents_recensés"]) * 100).round(2).fillna(0)

# Hazmat ou évacuation
hazmat = df_risk_valid[(df_risk_valid["Hazmat Cars Damaged"] > 0) | (df_risk_valid["Persons Evacuated"] > 0)]["Accident Type"].value_counts()
summary["% hazmat/evac"] = ((hazmat / summary["Incidents_recensés"]) * 100).round(2).fillna(0)

summary = summary.reset_index()

display(summary)


# Analyse de la vulnérabilité 

In [ ]:
# Fusion propre
df_risk = df_risk.merge(summary, how="left", on="Accident Type")

# Vérif rapide
print(df_risk.shape)
display(df_risk.head(3))

In [ ]:
# Evolution coûts, tués, blessés
annual_costs = df_risk_valid.groupby("Report Year")["Total Damage Cost"].sum()
annual_deaths = df_risk_valid.groupby("Report Year")["Total Persons Killed"].sum()
annual_injuries = df_risk_valid.groupby("Report Year")["Total Persons Injured"].sum()

fig, axs = plt.subplots(3, 1, figsize=(10, 12), sharex=True)
annual_costs.plot(ax=axs[0], title="Coût total annuel des accidents ($)", color='tab:orange')
annual_deaths.plot(ax=axs[1], title="Total annuel de tués")
annual_injuries.plot(ax=axs[2], title="Total annuel de blessés")
axs[2].set_xlabel("Année")
plt.tight_layout()
plt.show()


In [ ]:
# Coût moyen par type
cost_by_type = summary.set_index("Accident Type")["Coût_moyen_USD"].sort_values(ascending=False)
display(cost_by_type)
cost_by_type.plot(kind="bar", title="Coût moyen par type d'accident", figsize=(10,4))
plt.ylabel("Coût moyen ($)")
plt.show()

In [ ]:
# Ratio tués/blessés par incident
severity = summary[["Accident Type", "Tués_par_incident", "Blessés_par_incident"]].sort_values("Tués_par_incident", ascending=False)
display(severity)

In [ ]:
acc_types = df_risk_valid["Accident Type"].unique()
summary = pd.DataFrame({"Accident Type": acc_types}).set_index("Accident Type")

# Incidents
summary["Incidents_recensés"] = df_risk_valid["Accident Type"].value_counts()

# Fréquence = proba brute pour bornes min et max
summary["Frequence (%) (bas)"] = (summary["Incidents_recensés"] / total_trains_min * 100)
summary["Frequence (%) (haut)"] = (summary["Incidents_recensés"] / total_trains_max * 100)

# Conséquences brutes
summary["Coût_moyen_USD"] = df_risk_valid.groupby("Accident Type")["Total Damage Cost"].mean()
killed = df_risk_valid.groupby("Accident Type")["Total Persons Killed"].sum()
injured = df_risk_valid.groupby("Accident Type")["Total Persons Injured"].sum()
summary["Tués_par_incident"] = killed / summary["Incidents_recensés"]
summary["Blessés_par_incident"] = injured / summary["Incidents_recensés"]

# Normalisation de la conséquence seule
scaler = MinMaxScaler()
consequence_cols = ["Coût_moyen_USD", "Tués_par_incident", "Blessés_par_incident"]
summary[consequence_cols] = scaler.fit_transform(summary[consequence_cols].fillna(0))

# Conséquence agrégée 
summary["Consequence"] = (
    summary["Coût_moyen_USD"] * 0.5 +
    summary["Tués_par_incident"] * 0.3 +
    summary["Blessés_par_incident"] * 0.2
)

# Score final Risque = Frequence (bas) × Consequence
summary["Risque_composite"] = summary["Frequence (%) (bas)"] * summary["Consequence"]

# Criticité selon le score final
summary["Niveau_criticité"] = pd.qcut(
    summary["Risque_composite"],
    q=[0, 0.5, 0.75, 1],
    labels=["Low", "Medium", "High"]
)

# Reset index pour merge
summary = summary.reset_index()

# Merge complet
df_risk = df_risk.merge(
    summary[[
        "Accident Type",
        "Frequence (%) (bas)",
        "Frequence (%) (haut)",
        "Consequence",
        "Risque_composite",
        "Niveau_criticité"
    ]],
    on="Accident Type",
    how="left"
)

# Vérification
print(df_risk[[
    "Accident Type",
    "Frequence (%) (bas)",
    "Consequence",
    "Risque_composite",
    "Niveau_criticité"
]].head())

In [ ]:
# Supprimer tous les doublons
df_risk = df_risk.drop(
    columns=[
        'Risque_composite_x', 'Niveau_criticité_x',
        'Risque_composite_y', 'Niveau_criticité_y'
    ],
    errors='ignore'
)

# Vérifie la liste finale des colonnes
print("\nColonnes finales:")
print(df_risk.columns.tolist())

In [ ]:
print(f"Shape du df_risk : {df_risk.shape}")

describe_all = df_risk.describe().transpose()

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

display(describe_all.T)


In [ ]:
# Harmonisation des types d'accident
mapping_accident_type = {
    "Derailment": "Déraillement",
    "Hwy-rail crossing": "Collision voie-route",
    "RR grade crossing": "Collision voie-route",
    "Side collision": "Collision latérale",
    "Rear end collision": "Collision arrière",
    "Head on collision": "Collision frontale",
    "Broken train collision": "Collision convoi cassé",
    "Raking collision": "Collision latérale",
    "Fire/violent rupture": "Feu ou rupture",
    "Explosion-detonation": "Explosion",
    "Obstruction": "Obstacle sur voie",
    "Other (describe in narrative)": "Autre",
    "Other impacts": "Autre"
}
# Appliquer le mapping
df_risk["Accident Type"] = df_risk["Accident Type"].replace(mapping_accident_type)

In [ ]:
# Traduction du niveau de criticité
mapping_criticite = {
    "Low": "Faible",
    "Medium": "Modéré",
    "High": "Élevé"
}
df_risk["Niveau_criticité"] = df_risk["Niveau_criticité"].replace(mapping_criticite)

In [ ]:
# Traduction des tranches horaires
mapping_time_of_day = {
    "EARLY MORNING": "Tôt le matin",
    "LATE MORNING": "Fin de matinée",
    "AFTERNOON": "Après-midi",
    "EVENING": "Soirée"
}
df_risk["TimeOfDay"] = df_risk["TimeOfDay"].replace(mapping_time_of_day)

In [ ]:
# Définir les colonnes vraiment utiles
colonnes_utiles = [
    "Accident Number",
    "Accident Year", "Accident Month", "Day", "Report Year",
    "Accident Type", "State Abbreviation", "State Name", "County Name",
    "Latitude", "Longitude",
    "Weather Condition", "Visibility", "TimeOfDay",
    "Total Damage Cost", "Total Persons Killed", "Total Persons Injured",
    "Hazmat Cars", "Hazmat Cars Damaged", "Persons Evacuated",
    "Frequence (%) (bas)", "Frequence (%) (haut)",
    "Consequence", "Risque_composite", "Niveau_criticité"
]

# Extraire uniquement ces colonnes
df_final = df_risk[colonnes_utiles].copy()

# Export
df_final.to_csv("../data/cleaned/railroad_accident_cleaned.csv", index=False)
print("Fichier enregistré : /data/cleaned/railroad_accident_cleaned.csv")

## Synthèse de l’exploration et structuration du module Risque Ferroviaire

L’analyse exploratoire a permis de :

- Filtrer et nettoyer le jeu de données d’incidents ferroviaires pour isoler un sous-ensemble pertinent pour l’étude du risque logistique : variables structurantes, coûts, causes, conditions météo et indicateurs humains ont été conservés.

- Normaliser et convertir les colonnes clés pour garantir leur exploitabilité : traitement des valeurs manquantes, conversions en formats numériques cohérents, création de variables dérivées comme la tranche horaire (TimeOfDay).

- Analyser la distribution des accidents : fréquence par année, type, météo, tranche horaire ; répartition des coûts et impacts humains ; typologies d’incidents majeurs et incidents impliquant matières dangereuses.

- Estimer la probabilité d’occurrence de chaque type d’accident sur la période 2002–2022 en se basant sur une fourchette réaliste de trajets de trains par an.

- Calculer une conséquence normalisée par type d’accident en combinant coût moyen, tués et blessés par incident.

- Déterminer un indicateur de risque combiné comme produit de la fréquence et de la conséquence, et en déduire un niveau de criticité (Faible, Modéré, Élevé).

- Enrichir le jeu de données final avec ces métriques synthétiques pour chaque ligne d’incident, en plus d’indicateurs globaux sur la proportion d’incidents majeurs et de cas impliquant matières dangereuses ou évacuation.

Cette structuration garantit un module de risque ferroviaire exploitable pour l’analyse de la résilience de la chaîne logistique.
